In [17]:
from dotenv import dotenv_values
from collections import OrderedDict
import requests
import pandas as pd
import ast
import lxml

In [18]:
# OrderedDict containing key-value pairs of secret keys
config = dotenv_values("../.env")
CENSUS_API_KEY = config['CENSUS_API_KEY']

In [19]:
# API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
host = 'https://api.census.gov/data'
year = '/2019'
# Data Dictionary: https://api.census.gov/data.html
dataset_acronym = '/acs/acs1'
g = '?get='
# Variables for the ACS: https://api.census.gov/data/2005/acs/acs1/variables.html
variables = 'B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E'
location = '&for=us:*'
usr_key = f"&key={CENSUS_API_KEY}"

query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
response = requests.get(query_url)

In [20]:
response

<Response [200]>

B01001_001E is Estimated Total: Sex by Age without delineation.  In other words, this query is basically just returning 328,239,523 which is the total estimated US population in 2019.

Rather than going through and copying all the variable names from the reference table, I'm going to try and make things easier on myself and see if I can't just read that table in with pandas and extract the variable names.

In [7]:
variable_table_url = 'https://api.census.gov/data/2019/acs/acs1/variables.html'
v_table = pd.read_table(variable_table_url, skiprows=59)
v_table

/var/folders/4h/mbg_mjhs0gn8q3t8_ywpyvsh0000gr/T/ipykernel_64342/1726800111.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  v_table = pd.read_table(variable_table_url, skiprows=59)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,text-align: right;
0,NaN,NaN,NaN,NaN,NaN,NaN,padding-right: 1em;
1,NaN,NaN,NaN,NaN,NaN,NaN,width: 3em;
2,NaN,NaN,NaN,NaN,NaN,NaN,//font-size: smaller;
3,NaN,NaN,NaN,NaN,NaN,},NaN
4,.tagquery tr td { vertical...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
178279,</tr>,NaN,NaN,NaN,NaN,NaN,NaN
178280,</tbody>,NaN,NaN,NaN,NaN,NaN,NaN
178281,</table>,NaN,NaN,NaN,NaN,NaN,NaN
178282,</body>,NaN,NaN,NaN,NaN,NaN,NaN


Well, line 59 threw an error so I skipped it but the results aren't good. Because I'm too tired and lazy right now to figure out how to make that work properly, I'm going to give read_html a shot really quick.

In [4]:
variable_table_url = 'https://api.census.gov/data/2019/acs/acs1/variables.html'
v_table = pd.read_html(variable_table_url)
v_table

[                  Name                                       Label   
 0               AIANHH                                   Geography  \
 1                 ANRC                                   Geography   
 2          B01001_001E                            Estimate!!Total:   
 3          B01001_002E                     Estimate!!Total:!!Male:   
 4          B01001_003E      Estimate!!Total:!!Male:!!Under 5 years   
 ...                ...                                         ...   
 35551            STATE                                   Geography   
 35552         SUMLEVEL                          Summary Level code   
 35553               UA                                   Geography   
 35554            ucgid  Uniform Census Geography Identifier clause   
 35555  35555 variables                             35555 variables   
 
                                   Concept         Required   
 0                                     NaN     not required  \
 1                  

In [5]:
type(v_table)

list

In [6]:
variable_df = pd.DataFrame(v_table[0])
variable_df

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANHH,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
1,ANRC,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
2,B01001_001E,Estimate!!Total:,SEX BY AGE,not required,"B01001_001EA, B01001_001M, B01001_001MA",0,int,B01001,NaN
3,B01001_002E,Estimate!!Total:!!Male:,SEX BY AGE,not required,"B01001_002EA, B01001_002M, B01001_002MA",0,int,B01001,NaN
4,B01001_003E,Estimate!!Total:!!Male:!!Under 5 years,SEX BY AGE,not required,"B01001_003EA, B01001_003M, B01001_003MA",0,int,B01001,NaN
...,...,...,...,...,...,...,...,...,...
35551,STATE,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
35552,SUMLEVEL,Summary Level code,NaN,not required,NaN,0,string,NaN,NaN
35553,UA,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
35554,ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,predicate-only,NaN,0,ucgid,NaN,NaN


That's more like it!  This will make it easier to automate pulling out multiple variables and giving them more appropriate names than 'B01001_001E', for instance.

In [7]:
total_male_by_age_variables = ",".join(variable_df.iloc[3:27]['Name'].values)
total_male_by_age_variables

'B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E'

Ok, that gets me a string representation of all the variable names for the male population by age. I just picked those because they were at the top of the list.  I'm going to insert those into the API query and see what we get here.

In [8]:
# Only thing changing here is the variables which are substituted in under total_male_by_age_variables
m_query_url = f"{host}{year}{dataset_acronym}{g}{total_male_by_age_variables}{location}{usr_key}"

m_response = requests.get(m_query_url)

In [9]:
m_response.text

'[["B01001_002E","B01001_003E","B01001_004E","B01001_005E","B01001_006E","B01001_007E","B01001_008E","B01001_009E","B01001_010E","B01001_011E","B01001_012E","B01001_013E","B01001_014E","B01001_015E","B01001_016E","B01001_017E","B01001_018E","B01001_019E","B01001_020E","B01001_021E","B01001_022E","B01001_023E","B01001_024E","B01001_025E","us"],\n["161588973","9938937","10033518","10987313","6361859","4541794","2318283","2257008","6439169","11817829","11281470","10892040","10028675","10079567","10075795","10440265","4168435","5882735","3538792","4652319","6529918","4367764","2671396","2284092","1"]]'

So we really just want the second list since those will be the values.  We'll also want to use the 'label' column from the `variable_df` to get column headers that actually mean something. The last item in the `m_response.text[1]` is just the geography code for the US which is 1, so we'll drop that value as well.

In [12]:
m_values = [int(i) for i in ast.literal_eval(m_response.text)[1][:-1]]
m_values

[161588973,
 9938937,
 10033518,
 10987313,
 6361859,
 4541794,
 2318283,
 2257008,
 6439169,
 11817829,
 11281470,
 10892040,
 10028675,
 10079567,
 10075795,
 10440265,
 4168435,
 5882735,
 3538792,
 4652319,
 6529918,
 4367764,
 2671396,
 2284092]

We'll clean the labels.
Example: 'Estimate!!Total:!!Male:!!Under 5 years' -> 'Male: Under 5 year'

In [13]:
m_labels = ['Male: Total', *[i.strip('Estimate!!Total:!!').replace("!!", " ") for i in variable_df.iloc[4:27]['Label'].values]]
m_labels

['Male: Total',
 'Male: Under 5 year',
 'Male: 5 to 9 year',
 'Male: 10 to 14 year',
 'Male: 15 to 17 year',
 'Male: 18 and 19 year',
 'Male: 20 year',
 'Male: 21 year',
 'Male: 22 to 24 year',
 'Male: 25 to 29 year',
 'Male: 30 to 34 year',
 'Male: 35 to 39 year',
 'Male: 40 to 44 year',
 'Male: 45 to 49 year',
 'Male: 50 to 54 year',
 'Male: 55 to 59 year',
 'Male: 60 and 61 year',
 'Male: 62 to 64 year',
 'Male: 65 and 66 year',
 'Male: 67 to 69 year',
 'Male: 70 to 74 year',
 'Male: 75 to 79 year',
 'Male: 80 to 84 year',
 'Male: 85 years and over']

In [63]:
{m_labels[i]: m_values[i] for i in range(len(m_labels))}

{'Male: Total': 161588973,
 'Male: Under 5 year': 9938937,
 'Male: 5 to 9 year': 10033518,
 'Male: 10 to 14 year': 10987313,
 'Male: 15 to 17 year': 6361859,
 'Male: 18 and 19 year': 4541794,
 'Male: 20 year': 2318283,
 'Male: 21 year': 2257008,
 'Male: 22 to 24 year': 6439169,
 'Male: 25 to 29 year': 11817829,
 'Male: 30 to 34 year': 11281470,
 'Male: 35 to 39 year': 10892040,
 'Male: 40 to 44 year': 10028675,
 'Male: 45 to 49 year': 10079567,
 'Male: 50 to 54 year': 10075795,
 'Male: 55 to 59 year': 10440265,
 'Male: 60 and 61 year': 4168435,
 'Male: 62 to 64 year': 5882735,
 'Male: 65 and 66 year': 3538792,
 'Male: 67 to 69 year': 4652319,
 'Male: 70 to 74 year': 6529918,
 'Male: 75 to 79 year': 4367764,
 'Male: 80 to 84 year': 2671396,
 'Male: 85 years and over': 2284092}

In [14]:
pd.DataFrame({2019: {m_labels[i]: m_values[i] for i in range(len(m_labels))}}).reindex(m_labels)

,2019
Male: Total,161588973
Male: Under 5 year,9938937
Male: 5 to 9 year,10033518
Male: 10 to 14 year,10987313
Male: 15 to 17 year,6361859
Male: 18 and 19 year,4541794
Male: 20 year,2318283
Male: 21 year,2257008
Male: 22 to 24 year,6439169
Male: 25 to 29 year,11817829


___

Ok, there is all the male population information for 2019.  I'm going to try 2018 as well but I'm worried that the indexes of variables may have changed over the years.  We'll see how it goes. Before I do that I'm going to write some functions so I can just pop new info in without copying and pasting for every year now that I have a somewhat working proof of concept.

In [15]:
year = 2018

In [16]:
def get_variable_table_df(year):
    variable_table_url = f'https://api.census.gov/data/{year}/acs/acs1/variables.html'
    v_table = pd.read_html(variable_table_url)
    variable_df = pd.DataFrame(v_table[0])

    return variable_df

In [7]:
v_table = get_variable_table_df(year)
v_table

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANHH,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
1,ANRC,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
2,B00001_001E,Estimate!!Total,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,not required,B00001_001EA,0,int,B00001,NaN
3,B00002_001E,Estimate!!Total,UNWEIGHTED SAMPLE HOUSING UNITS,not required,B00002_001EA,0,int,B00002,NaN
4,B01001_001E,Estimate!!Total,SEX BY AGE,not required,"B01001_001EA, B01001_001M, B01001_001MA",0,int,B01001,NaN
...,...,...,...,...,...,...,...,...,...
35524,SDUNI,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
35525,STATE,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
35526,UA,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
35527,ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,predicate-only,NaN,0,ucgid,NaN,NaN


In [35]:
def get_male_by_age_index(variable_table):
    start_index = variable_table[((variable_table['Label'] == 'Estimate!!Total!!Male') | (variable_table['Label'] == 'Estimate!!Total:!!Male:')) & (variable_table['Concept'] == 'SEX BY AGE')].index[0]
    end_index = variable_table[((variable_table['Label'] == 'Estimate!!Total!!Male!!85 years and over') | (variable_table['Label'] == 'Estimate!!Total:!!Male:!!85 years and over')) & (variable_table['Concept'] == 'SEX BY AGE')].index[0]
    return start_index, end_index + 1

In [9]:
male_by_age_indeces = get_male_by_age_index(v_table)
male_by_age_indeces

(5, 29)

In [10]:
def get_variable_names(variable_table, indeces):

    total_male_by_age_variables = ",".join(variable_table.iloc[indeces[0]: indeces[1]]['Name'].values)
    return total_male_by_age_variables

In [11]:
variables = get_variable_names(v_table, male_by_age_indeces)
variables

'B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E'

In [12]:
def get_query_url(year, variables):
    # API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    # Data Dictionary: https://api.census.gov/data.html
    dataset_acronym = '/acs/acs1'
    g = '?get='
    # Variables for the ACS: https://api.census.gov/data/2005/acs/acs1/variables.html
    # variables = 'NAME,B01001_001E'
    location = '&for=us:*'
    usr_key = f"&key={USCensusAPI.api_key}"

    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"

    return query_url

In [13]:
query_url = get_query_url(year, variables)

In [14]:
def get_query_text(query_url):
    response = requests.get(query_url)
    return response.text

In [15]:
response_text = get_query_text(query_url)
response_text

'[["B01001_002E","B01001_003E","B01001_004E","B01001_005E","B01001_006E","B01001_007E","B01001_008E","B01001_009E","B01001_010E","B01001_011E","B01001_012E","B01001_013E","B01001_014E","B01001_015E","B01001_016E","B01001_017E","B01001_018E","B01001_019E","B01001_020E","B01001_021E","B01001_022E","B01001_023E","B01001_024E","B01001_025E","us"],\n["161118151","10047966","10095699","11000943","6394674","4578048","2334371","2261277","6526051","11869477","11111452","10790784","9900261","10253794","10266496","10515549","4134525","5745886","3437352","4618529","6221547","4195815","2591562","2226093","1"]]'

In [16]:
def get_values_from_response(response_text):
    values = [int(i) for i in ast.literal_eval(response_text)[1][:-1]]
    return values

In [17]:
vals = get_values_from_response(response_text)
vals

[161118151,
 10047966,
 10095699,
 11000943,
 6394674,
 4578048,
 2334371,
 2261277,
 6526051,
 11869477,
 11111452,
 10790784,
 9900261,
 10253794,
 10266496,
 10515549,
 4134525,
 5745886,
 3437352,
 4618529,
 6221547,
 4195815,
 2591562,
 2226093]

In [37]:
def get_labels(variable_df, indeces):
    # labels = ['Male: Total', *[i.strip('Estimate!!Total:!!').replace("!!", " ") for i in variable_df.iloc[indeces[0]:indeces[1]]['Label'].values]]
    labels = [i.replace("!!", " ").replace(":", "") for i in variable_df.iloc[indeces[0]:indeces[1]]['Label'].values]
    return labels

In [38]:
labels = get_labels(v_table, male_by_age_indeces)
labels

['Estimate Total Male',
 'Estimate Total Male Under 5 years',
 'Estimate Total Male 5 to 9 years',
 'Estimate Total Male 10 to 14 years',
 'Estimate Total Male 15 to 17 years',
 'Estimate Total Male 18 and 19 years',
 'Estimate Total Male 20 years',
 'Estimate Total Male 21 years',
 'Estimate Total Male 22 to 24 years',
 'Estimate Total Male 25 to 29 years',
 'Estimate Total Male 30 to 34 years',
 'Estimate Total Male 35 to 39 years',
 'Estimate Total Male 40 to 44 years',
 'Estimate Total Male 45 to 49 years',
 'Estimate Total Male 50 to 54 years',
 'Estimate Total Male 55 to 59 years',
 'Estimate Total Male 60 and 61 years',
 'Estimate Total Male 62 to 64 years',
 'Estimate Total Male 65 and 66 years',
 'Estimate Total Male 67 to 69 years',
 'Estimate Total Male 70 to 74 years',
 'Estimate Total Male 75 to 79 years',
 'Estimate Total Male 80 to 84 years',
 'Estimate Total Male 85 years and over']

In [20]:
def create_year_pop_dataframe(year, labels, values):
    df = pd.DataFrame({year: {labels[i]: values[i] for i in range(len(labels))}}).reindex(labels)
    return df

In [22]:
def create_male_pop_by_age_df(year):
    v_table = get_variable_table_df(year)
    male_by_age_indeces = get_male_by_age_index(v_table)
    variables = get_variable_names(v_table, male_by_age_indeces)
    query_url = get_query_url(year, variables)
    response_text = get_query_text(query_url)
    vals = get_values_from_response(response_text)
    labels = get_labels(v_table, male_by_age_indeces)
    df = create_year_pop_dataframe(year, labels, vals)
    return df

In [24]:
df_2018 = create_male_pop_by_age_df(2018)
df_2018

,2018
Estimate Total Male,161118151
Estimate Total Male Under 5 years,10047966
Estimate Total Male 5 to 9 years,10095699
Estimate Total Male 10 to 14 years,11000943
Estimate Total Male 15 to 17 years,6394674
Estimate Total Male 18 and 19 years,4578048
Estimate Total Male 20 years,2334371
Estimate Total Male 21 years,2261277
Estimate Total Male 22 to 24 years,6526051
Estimate Total Male 25 to 29 years,11869477


In [25]:
df_2017 = create_male_pop_by_age_df(2017)
df_2017

,2017
Estimate Total Male,160402504
Estimate Total Male Under 5 years,10138447
Estimate Total Male 5 to 9 years,10223398
Estimate Total Male 10 to 14 years,10854851
Estimate Total Male 15 to 17 years,6456448
Estimate Total Male 18 and 19 years,4547414
Estimate Total Male 20 years,2354931
Estimate Total Male 21 years,2284991
Estimate Total Male 22 to 24 years,6640221
Estimate Total Male 25 to 29 years,11720804


In [39]:
df_2019 = create_male_pop_by_age_df(2019)
df_2019

,2019
Estimate Total Male,161588973
Estimate Total Male Under 5 years,9938937
Estimate Total Male 5 to 9 years,10033518
Estimate Total Male 10 to 14 years,10987313
Estimate Total Male 15 to 17 years,6361859
Estimate Total Male 18 and 19 years,4541794
Estimate Total Male 20 years,2318283
Estimate Total Male 21 years,2257008
Estimate Total Male 22 to 24 years,6439169
Estimate Total Male 25 to 29 years,11817829


In [52]:
df = df_2017.merge(df_2018, left_index=True, right_index=True)
df = df.merge(df_2019, left_index=True, right_index=True)
df.T.reset_index().rename({'index': 'Year'}, axis=1)

,Year,Estimate Total Male,Estimate Total Male Under 5 years,Estimate Total Male 5 to 9 years,Estimate Total Male 10 to 14 years,Estimate Total Male 15 to 17 years,Estimate Total Male 18 and 19 years,Estimate Total Male 20 years,Estimate Total Male 21 years,Estimate Total Male 22 to 24 years,...,Estimate Total Male 50 to 54 years,Estimate Total Male 55 to 59 years,Estimate Total Male 60 and 61 years,Estimate Total Male 62 to 64 years,Estimate Total Male 65 and 66 years,Estimate Total Male 67 to 69 years,Estimate Total Male 70 to 74 years,Estimate Total Male 75 to 79 years,Estimate Total Male 80 to 84 years,Estimate Total Male 85 years and over
0,2017,160402504,10138447,10223398,10854851,6456448,4547414,2354931,2284991,6640221,...,10521449,10561492,4106765,5606148,3381427,4596961,5912776,3934609,2492574,2202102
1,2018,161118151,10047966,10095699,11000943,6394674,4578048,2334371,2261277,6526051,...,10266496,10515549,4134525,5745886,3437352,4618529,6221547,4195815,2591562,2226093
2,2019,161588973,9938937,10033518,10987313,6361859,4541794,2318283,2257008,6439169,...,10075795,10440265,4168435,5882735,3538792,4652319,6529918,4367764,2671396,2284092


In [53]:
df_2005 = create_male_pop_by_age_df(2005)

In [54]:
df_2005

,2005
Estimate Total Male,141274964
Estimate Total Male Under 5 years,10356773
Estimate Total Male 5 to 9 years,9945226
Estimate Total Male 10 to 14 years,10674272
Estimate Total Male 15 to 17 years,6415594
Estimate Total Male 18 and 19 years,3642423
Estimate Total Male 20 years,1946123
Estimate Total Male 21 years,1926099
Estimate Total Male 22 to 24 years,5861680
Estimate Total Male 25 to 29 years,9657845


In [63]:
from tqdm.notebook import tqdm
years = [i for i in range(2005, 2020)]
male_pop_by_age_df = pd.DataFrame()
for year in tqdm(years):
    try:
        y_df = create_male_pop_by_age_df(year)
        male_pop_by_age_df = male_pop_by_age_df.merge(y_df, left_index=True, right_index=True)
    except:
        next

  0%|          | 0/15 [00:00<?, ?it/s]

In [64]:
male_pop_by_age_df

,2005,2006,2007,2008,2009,2012,2013,2014,2015,2016,2017,2018,2019


Hmmm, did something wrong here ^^^^